In [213]:
import numpy as np
import pandas as pd
from datetime import datetime

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers

In [214]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [215]:
def hr_func(ts):
    date_time_obj = datetime.strptime(ts, '%Y-%m-%d %H:%M:%S')
    return date_time_obj.hour

train_data['time_hour'] = train_data['date_time'].apply(hr_func)
train_data = train_data.drop('date_time', axis=1)

In [216]:
test_data['time_hour'] = test_data['date_time'].apply(hr_func)
test_data.set_index('date_time')
#test_data = test_data.drop('date_time', axis=1)

,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,time_hour
date_time,,,,,,,,,
2011-01-01 00:00:00,8.0,41.3,0.4375,1108.8,745.7,797.1,880.0,1273.1,0
2011-01-01 01:00:00,5.1,51.7,0.4564,1249.5,864.9,687.9,972.8,1714.0,1
2011-01-01 02:00:00,5.8,51.5,0.4689,1102.6,878.0,693.7,941.9,1300.8,2
2011-01-01 03:00:00,5.0,52.3,0.4693,1139.7,916.2,725.6,1011.0,1283.0,3
2011-01-01 04:00:00,4.5,57.5,0.4650,1022.4,838.5,871.5,967.0,1142.3,4
...,...,...,...,...,...,...,...,...,...
2011-04-04 10:00:00,23.2,28.7,0.7568,1340.3,1023.9,522.8,1374.0,1659.8,10
2011-04-04 11:00:00,24.5,22.5,0.7119,1232.8,955.1,616.1,1226.1,1269.0,11
2011-04-04 12:00:00,26.6,19.0,0.6406,1187.7,1052.4,572.8,1253.4,1081.1,12


In [217]:
Y_train = train_data[['target_carbon_monoxide', 'target_benzene','target_nitrogen_oxides']].copy()
X_train = train_data.copy().drop(['target_carbon_monoxide', 'target_benzene','target_nitrogen_oxides'], axis = 1)

In [218]:
def dnn_model():
    model = keras.Sequential([
        layers.Input(shape = (9,)),
        layers.Dense(units =128, activation ='relu'),
        layers.LayerNormalization(),
        layers.Dropout(0.3),
        layers.Dense(3, activation = 'relu'),
    ])

    return model

In [219]:
model = dnn_model()
model.summary()



Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_39 (Dense)             (None, 128)               1280      
_________________________________________________________________
layer_normalization_20 (Laye (None, 128)               256       
_________________________________________________________________
dropout_20 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_40 (Dense)             (None, 3)                 387       
Total params: 1,923
Trainable params: 1,923
Non-trainable params: 0
_________________________________________________________________


In [220]:
early_stopping = callbacks.EarlyStopping(
    monitor="val_loss",
    patience=30,
    min_delta=0.0000001,
    restore_best_weights=True,
)

In [221]:
kfold = KFold(n_splits=5, shuffle=True)
#X_train = X_train.to_numpy()

for train_index, test_index in kfold.split(X_train, Y_train):
    model.compile(loss='mean_squared_logarithmic_error', optimizer = keras.optimizers.Adam(learning_rate=0.00005))
    history = model.fit(X_train.iloc[train_index], Y_train.iloc[train_index],
          batch_size = None, epochs = 150,
          validation_data=(X_train.iloc[test_index], Y_train.iloc[test_index]),
          #callbacks=[early_stopping],
           )
    score = model.evaluate(X_train.iloc[test_index], Y_train.iloc[test_index], verbose = 0)
    print('Test loss: {}'.format(score))
    print('Test loss sqrt: {}'.format(np.sqrt(score)))

Train on 5688 samples, validate on 1423 samples
Epoch 1/150
5688/5688 [==============================] - 1s 95us/sample - loss: 8.9594 - val_loss: 6.9341
Epoch 2/150
5688/5688 [==============================] - 0s 41us/sample - loss: 6.1854 - val_loss: 5.5274
Epoch 3/150
5688/5688 [===========================

In [222]:
test_data_copy = test_data.copy().drop('date_time', axis=1)
predicts = model.predict(test_data_copy)


In [223]:
output = pd.DataFrame(predicts, columns = ['target_carbon_monoxide', 'target_benzene','target_nitrogen_oxides'])
output['date_time'] = test_data['date_time']
output.to_csv('submission.csv', index=False)
